In [1]:
import elm
import numpy as np
import scipy
import pickle
import scipy.io

In [2]:
def getmax(L):
    a = 0
    b = 0
    c = 0
    for i in L:
        if i == 0:
            a += 1
        elif i == 1:
            b += 1
        else:
            c += 1
    if a > b >= c or a > c >= b:
        return 0
    if b > a >= c or b > c >= a:
        return 1
    if c > a >= b or c > b >= a:
        return 2
    return -1

def load_label(fromIndex, toIndex):
    label = scipy.io.loadmat("./data/SEED1/processed_data/label.mat")["label"]
    return label[0][fromIndex:toIndex]+1

def load(time_index, person_index, movie_st, movie_end):
    labellist = load_label(movie_st, movie_end) 
    data = []
    labels = []
    sample_number = []
    for loop_person in range(person_index-1, person_index):
        person_name = 'person%d' % (loop_person + 1)
        for loop_time in range(time_index-1, time_index):
            name_time = '_time%d' % (loop_time + 1)
            filename = person_name + name_time + '.mat'
            filedata = scipy.io.loadmat("./data/SEED1/processed_data/data/" + filename)
                
            for index in range(movie_st, movie_end):
                label = labellist[index - movie_st]
                feats = filedata['de_LDS' + str(index+1)]
                feats = feats.transpose()

                for j in range(feats.shape[1]):
                    data1 = []
                    for i in range(feats.shape[0]):
                        data1 = np.append(data1, feats[i][j])
                    data.append(data1)
                    labels.append(label)
                sample_number.append(feats.shape[1])
    sample_number = np.array(sample_number).T
    dataset = np.array(data)
    labels = np.array(labels).reshape(len(labels), 1)
    d = np.concatenate((labels, dataset), axis = 1)
    return d, sample_number

def loadmat(filename):
    return scipy.io.loadmat(filename)['YYM_H']

def unpickle(file):
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

def loadnpy(filename):
    return unpickle(filename)

In [11]:
for jjj in range(1):
    acc = []
    for _time in range(1, 4):    
        for loop in range(1, 15):
            Training, _ = load(_time, loop, 0, 9)
            Testing, sample_number = load(_time, loop, 9, 15)

            name = './data/SEED1/features/yym5_channel_%d_%d' % (_time, loop)
            
            YYM_H = loadmat(name+'feature_1.mat')
            train1 = YYM_H[:, :Training.shape[0]]
            test1 = YYM_H[:, Training.shape[0]:]
            Training = np.concatenate((Training[:, :1], train1.T), axis = 1)
            Testing = np.concatenate((Testing[:, :1], test1.T), axis=1)

            sample_acc = []
            acc1 = []
            yacc = []
            for ccc in range(1, 20):
                c2 = ccc-10
                
                C2 = 2 ** c2
                
                model = elm.ELM(1, 1, 'sig', 1, C2)
                _, Testing_acc, le, la = model.train(Training, Testing)
                
                sample = 0
                idx_expected = []
                idx_actural = []
                for i in range(sample_number.shape[0]):
                    A = le[sample:sample_number[i] + sample]
                    idx_expected.append(getmax(A))

                    B = la[sample:sample_number[i] + sample]
                    idx_actural.append(getmax(B))

                    sample += sample_number[i]

                idx_expected = np.array(idx_expected)
                idx_actural = np.array(idx_actural)

                accuracy = np.sum(idx_expected - idx_actural == 0) / 6
                acc1.append(accuracy)  
                sample_acc.append(Testing_acc)
                yacc.append(accuracy)
            acc.append(np.max(yacc))
            print(np.max(yacc), Testing_acc)
    total_test_accuarcy = np.mean(acc)
    print(total_test_accuarcy)

1.0 0.9161849710982659
0.8333333333333334 0.7167630057803468
1.0 0.940028901734104
0.8333333333333334 0.7658959537572254
0.8333333333333334 0.8591040462427746
0.8333333333333334 0.8323699421965318
1.0 0.819364161849711
1.0 0.8973988439306358
0.6666666666666666 0.6336705202312138
0.8333333333333334 0.8070809248554913
1.0 0.8280346820809248
0.8333333333333334 0.7391618497109826
1.0 0.7716763005780347
1.0 1.0
0.8333333333333334 0.6625722543352601
0.8333333333333334 0.8316473988439306
0.8333333333333334 0.6625722543352601
1.0 0.9183526011560694
0.6666666666666666 0.6828034682080926
1.0 0.828757225433526
1.0 0.7825144508670521
0.8333333333333334 0.8865606936416185
0.8333333333333334 0.6170520231213873
1.0 0.8692196531791907
0.8333333333333334 0.6748554913294798
1.0 0.7167630057803468
1.0 0.809971098265896
1.0 1.0
1.0 0.6156069364161849
1.0 1.0
0.8333333333333334 0.7355491329479769
0.8333333333333334 0.7203757225433526
1.0 0.9024566473988439
0.8333333333333334 0.5534682080924855
1.0 0.890173

In [4]:
for jjj in range(1):
    acc = []
    for _time in range(1, 4):    
        for loop in range(1, 15):
            Training, _ = load(_time, loop, 0, 9)
            Testing, sample_number = load(_time, loop, 9, 15)

            name = './data/SEED1/features/yym5_channel_%d_%d' % (_time, loop)
            
            YYM_H = loadmat(name+'feature_1.mat')
            train1 = YYM_H[:, :Training.shape[0]]
            test1 = YYM_H[:, Training.shape[0]:]
            train1 = np.concatenate((Training[:, :1], train1.T), axis = 1)
            test1 = np.concatenate((Testing[:, :1], test1.T), axis=1)

            train2 = loadnpy('./data/SEED1/feature2/train_' + str(_time) + "_" + str(loop))['data']
            test2 = loadnpy('./data/SEED1/feature2/test_' + str(_time) + "_" + str(loop))['data']
            #train2 = np.concatenate((Training[:, :1], train2), axis = 1)
            #test2 = np.concatenate((Testing[:, :1], test2), axis=1)
            
            Training = np.concatenate((train1, train2), axis=1)
            Testing = np.concatenate((test1, test2), axis=1)
            sample_acc = []
            acc1 = []
            yacc = []
            for ccc in range(1, 20):
                c2 = ccc-10
                
                C2 = 2 ** c2
                
                model = elm.ELM(1, 1, 'sig', 1, C2)
                _, Testing_acc, le, la = model.train(Training, Testing)
                
                sample = 0
                idx_expected = []
                idx_actural = []
                for i in range(sample_number.shape[0]):
                    A = le[sample:sample_number[i] + sample]
                    idx_expected.append(getmax(A))

                    B = la[sample:sample_number[i] + sample]
                    idx_actural.append(getmax(B))

                    sample += sample_number[i]

                idx_expected = np.array(idx_expected)
                idx_actural = np.array(idx_actural)

                accuracy = np.sum(idx_expected - idx_actural == 0) / 6
                acc1.append(accuracy)  
                sample_acc.append(Testing_acc)
                yacc.append(accuracy)
            acc.append(np.max(yacc))
            print(np.max(yacc), Testing_acc)
    total_test_accuarcy = np.mean(acc)
    print(total_test_accuarcy)

0.8333333333333334 0.5375722543352601
0.8333333333333334 0.8085260115606936
1.0 0.926300578034682
0.5 0.5859826589595376
1.0 0.8186416184971098
0.8333333333333334 0.6697976878612717
0.8333333333333334 0.7507225433526011
1.0 0.8410404624277457
0.6666666666666666 0.7160404624277457
0.8333333333333334 0.7044797687861272
1.0 0.9458092485549133
0.6666666666666666 0.7109826589595376
0.8333333333333334 0.6914739884393064
0.8333333333333334 0.5108381502890174
0.8333333333333334 0.6206647398843931
0.8333333333333334 0.7023121387283238
0.8333333333333334 0.7420520231213873
0.8333333333333334 0.6828034682080926
0.6666666666666666 0.6828034682080926
0.8333333333333334 0.8157514450867052
1.0 0.8395953757225434
1.0 0.8482658959537572
0.8333333333333334 0.5794797687861272
1.0 0.8988439306358381
0.6666666666666666 0.6336705202312138
0.6666666666666666 0.666907514450867
1.0 0.6856936416184971
1.0 0.953757225433526
0.8333333333333334 0.4725433526011561
1.0 0.8728323699421965
1.0 0.523121387283237
0.6666